# Preprocessing of Data

_Below is a function to count the Categorical and Numerical columns_

In [1]:
import time
start = time.time()
# Creating a function that will track quantitative and qualitative data
def count_cols(dataframe):
    '''
    This function returns the counts of numeric and categorical data columns. 
    '''
    
    # creating dataframe with numeric data
    quantitative_cols = dataframe.select_dtypes(include=[np.number])

    # creating dataframe with categorical data
    qualitative_cols = dataframe.select_dtypes(exclude=[np.number])

    # printing the counts of numeric and categorical data
    return [f'There are {quantitative_cols.shape[1]} numeric columns and \
{qualitative_cols.shape[1]} categorical columns.', quantitative_cols, qualitative_cols]

As this is a test dataset we wont change any true value here or drop any rows, we will just transform the dataset into required feature for testing the model, we will process the data by filling some NA values and transforming few skewed data as we did for training set. We are doing this because our model is trained on the instruction provided as input features, so that is what we also require in test set.

In [2]:
# Start with importing required library
import pandas as pd
import numpy as np

# read the dataset!
df = pd.read_csv('assets/data/Housing_Price_Test.csv')

In [3]:
# Importing Pacakges
import pandas as pd
import numpy as np
import warnings

# Configuring libs
warnings.filterwarnings('ignore')

print('Dependencies has been imported successfully.')

# Reading Dataset

# Test Data
df_test = pd.read_csv(r'assets/data/Housing_Price_Test.csv')
df_test.head()
print(f'Testing Dataset has been read successfully.\nRows : {df_test.shape[0]}\nColumns : {df_test.shape[1]}')

# Checking for the presence of duplicate entry
df_test.duplicated('Id').sum()


# Deleting 'Id' Column
del df_test['Id']

# Replacing Nan value

# For below column with missing values we have an option to replace the missing values wiht 'NA'
# as 'NA' is a choice available for readings of those columns
# So, we consider that 'Nan' values was something that wasn't applicable for the respective house.
for col in ['Alley', 'PoolQC', 'MiscFeature', 'Fence', 'FireplaceQu', 'GarageCond', 'GarageType', 'GarageFinish', 'GarageQual', 'BsmtExposure', 'BsmtFinType2', 'BsmtCond', 'BsmtQual', 'BsmtFinType1', ]:
    df_test[col].fillna('NA', inplace=True)
    
# Similar to previous case, this column has choice 'None' equivalent to 'NA'
# So, we consider to replace it with 'None'
df_test['MasVnrType'].fillna('None', inplace=True)

# Filling 'Nan' values of below listed column with mean
df_test['LotFrontage'] = df_test.groupby('Street')['LotFrontage'].transform(lambda x:x.fillna(x.mean()))
df_test['MasVnrArea'] = df_test.groupby('MasVnrType')['MasVnrArea'].transform(lambda x:x.fillna(x.mean()))
df_test['GarageYrBlt'] = df_test.groupby('GarageQual')['GarageYrBlt'].transform(lambda x:x.fillna(x.mode()))

# Converting behaviour of 'MoSold' from numerical to categorical
df_test['MoSold'] = df_test['MoSold'].apply(lambda x: str(x))

# Lets add some new columns into dataframe using previous columns.<br>
# For e.g. :
#    - HouseAge & GarageAge : This will define how many year old the house & garage is.
#    - TotalBath : We can sum up the total bathroom available in the house.
#    - 2FlrBld : It tells whether the house has 2 floor or not.
#      and many other.

# Adding new columns

# How many year old the houses are ?
df_test['HouseAge'] = df_test['YrSold'] - df_test['YearRemodAdd'] 

# How many year old the garages are ?
df_test['GarageAge'] = df_test['YrSold'] - df_test['GarageYrBlt']
# Replacing leftover 'Nan' values with 0, for the houses that do not have garage
df_test['GarageAge'].fillna(0, inplace=True) 

# Consdering Total Bathrooms Available in the house
df_test['TotalBath'] = df_test['BsmtFullBath'] + df_test['FullBath'] + df_test['BsmtHalfBath']*0.5 + df_test['HalfBath']*0.5

# Considering Total Porch Square Ft.
df_test['TotalPorchSF'] = df_test['OpenPorchSF'] + df_test['EnclosedPorch'] + df_test['3SsnPorch'] + df_test['ScreenPorch']

# Checking if house has Multiple Floor
df_test['2FlrBld'] = df_test['2ndFlrSF'].apply(lambda x: 0 if x==0 else 1)

# Checking if Basement is Available with house
df_test['BsmtAvailable'] = df_test['BsmtExposure'].apply(lambda x: 0 if x=='NA' else 1)

# As we have used some column to create new column, now we can delete the old columns.<br />
# In below cell we will drop the columns that isn't required<br />

# List of column names need to be dropped as non required / repetational
drop_col = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
            '1stFlrSF', '2ndFlrSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
            'HalfBath', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch',
            'YrSold', 'BsmtExposure', 'Electrical']

# List of column names for which dummy var is to be created
dummy_col = ['MoSold', 'GarageType', 
             'GarageFinish', 'BsmtFinType2', 'BsmtFinType1']

# Droping columns mentioned in `drop_col` variable
df_test.drop(drop_col, axis=1, inplace=True)

# Dropping column with leftover Nan value (Only one row with Nan in Electrical Column!) 
df_test.dropna(inplace=True)

# Reseting the index number after dropping all the missing values
df_test.reset_index(drop=True, inplace=True)

new_df = pd.concat([df_test, pd.get_dummies(df_test[dummy_col], drop_first=True)], axis=1)
new_df.drop(dummy_col, axis=1, inplace=True)

# ## Feature Selection (Quantitaive Feature)
# Dropping column with Multi-colinearity
new_df.drop(['GarageAge', 'GarageArea'], axis=1, inplace=True)

# ## Normalizing data

from scipy.stats import norm

# After Normalizing
new_df['LotFrontage'] = np.log(new_df.LotFrontage)

new_df['GrLivArea'] = np.log(new_df.GrLivArea)

# drop pearson non significant columns
new_df.drop(['MSSubClass', 'OverallCond', 'LowQualFinSF', 'PoolArea', 'MiscVal'], axis=1, inplace=True)

# ## ANOVA
# 
# Analysis of variance, or ANOVA, is a powerful statistical technique that involves partitioning the observed variance into different components to conduct various significance tests.
drop_anova = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterCond', 'BsmtCond', 'Heating', 'Functional', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
new_df.drop(drop_anova, axis=1, inplace=True)
create_qual_dummies = ['Neighborhood', 'MasVnrType', 'ExterQual', 'Foundation', 'BsmtQual', 'HeatingQC', 'CentralAir', 'KitchenQual', 'FireplaceQu']

gradeof5 = {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}
gradeof6 = {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}

for col in ['ExterQual', 'HeatingQC', 'KitchenQual']:
    new_df[col].replace(gradeof5, inplace=True)
    create_qual_dummies.remove(col)

for col in ['FireplaceQu', 'BsmtQual']:
    new_df[col].replace(gradeof6, inplace=True)
    create_qual_dummies.remove(col)

new_df = pd.concat([new_df,pd.get_dummies(new_df[create_qual_dummies], drop_first=True)], axis=1)
new_df.drop(create_qual_dummies, axis=1, inplace=True)


new_df.to_csv('output/dataset/Housing clean test_dataset.csv')

count_cols(new_df)[0]

Dependencies has been imported successfully.
Testing Dataset has been read successfully.
Rows : 1459
Columns : 80


'There are 87 numeric columns and 0 categorical columns.'

The execution time recorded for the test data transformation was `1.03 second`.

<h3>Author</h3>
<h4>Akash Sharma</h4>
<div style="float:left">
  <a href="https://www.linkedin.com/in/akash-sharma-01775b14a">
    <img src="https://img.shields.io/badge/LinkedIn-0077B5?style=for-the-badge&logo=linkedin&logoColor=white" alt="LinkedIn">
  </a>
  <a href="https://discord.com/users/366283102462541865">
    <img src="https://img.shields.io/badge/Discord-7289DA?style=for-the-badge&logo=discord&logoColor=white" alt="Discord">
  </a>
  <a href="https://github.com/CosmiX-6">
    <img src="https://img.shields.io/badge/GitHub-100000?style=for-the-badge&logo=github&logoColor=white" alt="GitHub">
  </a>
</div>